In [3]:
import os

In [4]:
root_path = os.path.abspath("../")
proc_data_path = os.path.join(root_path, "data", "processed")
inter_data_path = os.path.join(root_path, "data", "interim")

HUGGING FACE

In [4]:
from datasets import load_dataset

In [5]:
hg_data = load_dataset("tiny_shakespeare")
hg_train_text = hg_data["train"]["text"][0]
hg_val_text = hg_data["validation"]["text"][0]
hg_all_text = f"{hg_train_text} {hg_val_text}"
hg_base_path = os.path.join(proc_data_path, "HuggingFace", "base")

open(os.path.join(hg_base_path, "base.txt"), "w+").write(hg_all_text)
open(os.path.join(hg_base_path, "train.txt"), "w+").write(hg_train_text)
open(os.path.join(hg_base_path, "val.txt"), "w+").write(hg_val_text)

Found cached dataset tiny_shakespeare (C:/Users/willi/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

55770

In [6]:
def write_sequence_data(input_text, base_path, seq_len, f_name
                        ,input_splitter = ""):
    seq_path = os.path.join(base_path, f"Seq{seq_len}")
    if not os.path.exists(seq_path): os.makedirs(seq_path)
    seq_count = int(len(input_text)/seq_len)

    sting_splits = [f"{input_splitter}".join(input_text[i*seq_len:(i+1)*seq_len]) + "[SEQ_SPLITTER]"
     for i in range(seq_count)]
    
    open(os.path.join(seq_path, f_name), "w+").write(''.join(sting_splits))
    

In [7]:
hg_all_data_path = os.path.join(proc_data_path, "HuggingFace")
seq_lens = [50,100,150,200,300,500,1000,3000,5000]

In [8]:
for seq_len_given in seq_lens:
    write_sequence_data(hg_all_text, hg_all_data_path, seq_len_given, "base.txt")
    write_sequence_data(hg_train_text, hg_all_data_path, seq_len_given, "train.txt")
    write_sequence_data(hg_val_text, hg_all_data_path, seq_len_given, "val.txt")

LARGER TEXT DATASET

In [9]:
large_data_path = os.path.join(proc_data_path, "LargerText")
large_all_text = open(os.path.join(large_data_path,"base.txt"), "r").read()
file_len = len(large_all_text)
train_len = int(file_len * 0.9)
large_train_text = large_all_text[:train_len]
large_val_text = large_all_text[train_len:]
print(f"{len(large_all_text)}, {len(large_train_text)}, {len(large_val_text)}")

4357781, 3922002, 435779


In [10]:
open(os.path.join(large_data_path, "train.txt"), "w+").write(large_train_text)
open(os.path.join(large_data_path, "val.txt"), "w+").write(large_val_text)

435779

In [11]:
for seq_len_given in seq_lens:
    write_sequence_data(hg_all_text, large_data_path, seq_len_given, "base.txt")
    write_sequence_data(hg_train_text, large_data_path, seq_len_given, "train.txt")
    write_sequence_data(hg_val_text, large_data_path, seq_len_given, "val.txt")

WEBSCRAPE DATA

In [22]:
import pickle

In [23]:
def get_webscrape_data(data_path: str):
    all_eng_text = []
    all_og_text =[]

    for dir_eg in os.listdir(data_path):
        path_to_play = os.path.join(data_path, dir_eg)
        
        with open(os.path.join(path_to_play, "english_lines.txt"), "rb") as fp:   # Unpickling
            play_eng_lines = pickle.load(fp)

        with open(os.path.join(path_to_play, "og_lines.txt"), "rb") as fp2:   # Unpickling
            play_og_lines = pickle.load(fp2)

        if len(play_eng_lines) != len(play_og_lines): print("PROBLEMS")

        all_eng_text += play_eng_lines
        all_og_text += play_og_lines

    return all_eng_text, all_og_text#[:5000]

In [24]:
web_scrape_inter_base = os.path.join(inter_data_path, "webdata")

In [16]:
import re

In [42]:
all_eng_text, all_og_text = get_webscrape_data(web_scrape_inter_base)
print(len(all_eng_text))
print(len(all_og_text))

all_eng_text = [re.sub(' {2,}', ' ', x.replace(" ' ", "'").replace("\n", " ").strip()) + "\n" for x in all_eng_text]
all_og_text = [re.sub(' {2,}', ' ', x.replace(" ' ", "'").strip()) + "\n" for x in all_og_text]

print(len(all_eng_text))
print(len(all_og_text))

32159
32159
32159
32159


In [43]:
webscrape_data_path = os.path.join(proc_data_path, "Webscrape")

In [44]:
open(os.path.join(webscrape_data_path, "base", "eng_base.txt"), "w+").write(' '.join(all_eng_text))
open(os.path.join(webscrape_data_path, "base", "og_base.txt"), "w+").write(' '.join(all_og_text))

4449977

In [45]:
training_count = int(len(all_eng_text) * 0.9)

In [46]:
seq_for_trans = 1

In [47]:
def write_sequence_data(input_text, base_path, seq_len, f_name
                        ,input_splitter = ""):
    seq_path = os.path.join(base_path, f"Seq{seq_len}")
    if not os.path.exists(seq_path): os.makedirs(seq_path)
    seq_count = int(len(input_text)/seq_len)

    sting_splits = ["* " + f"{input_splitter}".join(input_text[i*seq_len:(i+1)*seq_len]) + "* [SEQ_SPLITTER]"
     for i in range(seq_count)]
    
    open(os.path.join(seq_path, f_name), "w+").write(''.join(sting_splits))

In [48]:
write_sequence_data(all_eng_text, webscrape_data_path, seq_for_trans, "context_base.txt")
write_sequence_data(all_eng_text[:training_count], webscrape_data_path, seq_for_trans, "context_train.txt")
write_sequence_data(all_eng_text[training_count:], webscrape_data_path, seq_for_trans, "context_val.txt")

write_sequence_data(all_og_text, webscrape_data_path, seq_for_trans, "content_base.txt")
write_sequence_data(all_og_text[:training_count], webscrape_data_path, seq_for_trans, "content_train.txt")
write_sequence_data(all_og_text[training_count:], webscrape_data_path, seq_for_trans, "content_val.txt")

In [31]:
print(len(all_eng_text))
print(len(all_eng_text[:training_count]))
print(len(all_eng_text[training_count:]))

print(len(all_og_text))
print(len(all_og_text[:training_count]))
print(len(all_og_text[training_count:]))

32159
28943
3216
32159
28943
3216


In [5]:
def get_webscrape_data(data_path: str):
    all_eng_text = []
    all_og_text =[]

    for dir_eg in os.listdir(data_path):
        path_to_play = os.path.join(data_path, dir_eg)
        
        with open(os.path.join(path_to_play, "english_lines.txt"), "rb") as fp:   # Unpickling
            play_eng_lines = pickle.load(fp)

        with open(os.path.join(path_to_play, "og_lines.txt"), "rb") as fp2:   # Unpickling
            play_og_lines = pickle.load(fp2)

        if len(play_eng_lines) != len(play_og_lines): print("PROBLEMS")

        all_eng_text += play_eng_lines
        all_og_text += play_og_lines

    return all_eng_text, all_og_text#[:5000]

In [6]:
web_scrape_inter_base = os.path.join(inter_data_path, "webdata")

In [9]:
import pickle

In [10]:
all_eng_text, all_og_text = get_webscrape_data(web_scrape_inter_base)

In [15]:
print(len(all_eng_text))
print(len(all_og_text))

32159
32159


In [13]:
print(all_eng_text[8783])
print(all_og_text[8783])


Hark you, the king is coming, and I must speak with himfrom the pridge.—God pless your Majesty.
